In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Defining residual blocks
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += self.shortcut(residual)
        out = self.relu(out)
        return out

# Defining Lightweight ResNet
class LightResNet(nn.Module):
    def __init__(self, num_classes=10):
        super(LightResNet, self).__init__()
        self.in_channels = 64

        # Initial Convolutional Layer
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)

        # sequence of residual blocks
        self.layer1 = self._make_layer(64, 2, stride=1)
        self.layer2 = self._make_layer(128, 2, stride=2)
        self.layer3 = self._make_layer(256, 2, stride=2)

        # Global average pooling and full connectivity
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(256, num_classes)

    def _make_layer(self, out_channels, num_blocks, stride):
        layers = []
        layers.append(ResidualBlock(self.in_channels, out_channels, stride))
        self.in_channels = out_channels
        for _ in range(1, num_blocks):
            layers.append(ResidualBlock(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        x = self.avg_pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# Print model parameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LightResNet().to(device)

# Data enhancement and loading
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # data enhancement
    transforms.RandomCrop(32, padding=4),  # Output size 32 x 32
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),  # Color Jitter
    transforms.RandomRotation(15),  # random rotation
    transforms.ToTensor(), # Convert to Tensor + auto-normalize to [0,1]
    transforms.Normalize((0.4914, 0.4822, 0.4465),  # normalized mean
                         (0.2023, 0.1994, 0.2010)), # normalized standard deviation
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.1), ratio=(0.3, 3.3))  # Cutout
])

transform_test = transforms.Compose([
    transforms.ToTensor(),  # Convert to Tensor + auto-normalize to [0,1]
    transforms.Normalize((0.4914, 0.4822, 0.4465),   # standardization
                         (0.2023, 0.1994, 0.2010))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False)

# Training configuration
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100)



# Initialize Logger
train_losses = []  # Record the training loss for each epoch
test_accuracies = []  # Record test accuracy for each epoch

# Modified training function (training loss print added)
def train(epoch):
    model.train()
    total_loss = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        # Print batch training progress in real time
        if batch_idx % 50 == 0:  # Prints every 50 batch
            current_loss = loss.item()
            current_batch = batch_idx + 1
            total_batches = len(trainloader)
            print(f'\rEpoch: {epoch} | '
                  f'Progress: [{current_batch}/{total_batches}] | '
                  f'Batch Loss: {current_loss:.4f}', end='', flush=True)

    # Print the average training loss at the end of each epoch
    avg_loss = total_loss / len(trainloader)
    train_losses.append(avg_loss)  # Recording training losses
    print(f'\nEpoch: {epoch} | '
          f'Train Loss: {avg_loss:.4f} | ', end='')

# test function (add accuracy print)
def test():
    model.eval()
    correct = 0
    total = 0
    test_loss = 0
    with torch.no_grad():
        for inputs, targets in testloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    acc = 100. * correct / total
    avg_test_loss = test_loss / len(testloader)
    test_accuracies.append(acc)  # Record test accuracy
    print(f'Test Loss: {avg_test_loss:.4f} | '
          f'Test Acc: {acc:.2f}%')
    return acc

# Modified training loop (add progress bar style output)
for epoch in range(100):
    # training phase
    train(epoch)
    scheduler.step()

    # Tested every 5 epochs
    if epoch % 5 == 0 or epoch == 199:  # Final epoch forced test
        test_acc = test()

    # Stage Summary Print
    if (epoch + 1) % 10 == 0:
        print('-'*80)
        print(f'[Phase Summary] Epoch: {epoch} | '
              f'Latest Train Loss: {train_losses[-1]:.4f} | '
              f'Latest Test Acc: {test_accuracies[-1]:.2f}%')
        print('-'*80)

# save the model
torch.save(model.state_dict(), 'cifar10_resnet.pth')

100%|██████████| 170M/170M [00:05<00:00, 30.9MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Epoch: 0 | Progress: [351/391] | Batch Loss: 1.4270
Epoch: 0 | Train Loss: 1.5763 | Test Loss: 1.6025 | Test Acc: 50.32%
Epoch: 1 | Progress: [351/391] | Batch Loss: 1.0468
Epoch: 2 | Progress: [351/391] | Batch Loss: 0.9700
Epoch: 3 | Progress: [351/391] | Batch Loss: 0.8007
Epoch: 4 | Progress: [351/391] | Batch Loss: 0.7561
Epoch: 5 | Progress: [351/391] | Batch Loss: 0.8172
Epoch: 5 | Train Loss: 0.6841 | Test Loss: 0.7057 | Test Acc: 77.47%
Epoch: 6 | Progress: [351/391] | Batch Loss: 0.6822
Epoch: 7 | Progress: [351/391] | Batch Loss: 0.6114
Epoch: 8 | Progress: [351/391] | Batch Loss: 0.5487
Epoch: 9 | Progress: [351/391] | Batch Loss: 0.3403
Epoch: 9 | Train Loss: 0.5354 | --------------------------------------------------------------------------------
[阶段总结] Epoch: 9 | Latest Train Loss: 0.5354 | Latest Test Acc: 77.47%
----------------------------------------------------------------------

In [ ]:
import torch
import torchvision.transforms as transforms
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader

class ResidualBlock(torch.nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = torch.nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = torch.nn.BatchNorm2d(out_channels)
        self.relu = torch.nn.ReLU(inplace=True)
        self.conv2 = torch.nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = torch.nn.BatchNorm2d(out_channels)

        self.shortcut = torch.nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = torch.nn.Sequential(
                torch.nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                torch.nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += self.shortcut(residual)
        out = self.relu(out)
        return out

class LightResNet(torch.nn.Module):
    def __init__(self, num_classes=10):
        super(LightResNet, self).__init__()
        self.in_channels = 64

        self.conv1 = torch.nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = torch.nn.BatchNorm2d(64)
        self.relu = torch.nn.ReLU(inplace=True)

        self.layer1 = self._make_layer(64, 2, stride=1)
        self.layer2 = self._make_layer(128, 2, stride=2)
        self.layer3 = self._make_layer(256, 2, stride=2)

        self.avg_pool = torch.nn.AdaptiveAvgPool2d((1, 1))
        self.fc = torch.nn.Linear(256, num_classes)

    def _make_layer(self, out_channels, num_blocks, stride):
        layers = []
        layers.append(ResidualBlock(self.in_channels, out_channels, stride))
        self.in_channels = out_channels
        for _ in range(1, num_blocks):
            layers.append(ResidualBlock(out_channels, out_channels))
        return torch.nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        x = self.avg_pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

from google.colab import drive
drive.mount('/content/drive')

def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict


def load_test_data():
    test_data_dict = unpickle("cifar_test_nolabel.pkl")
    images = test_data_dict[b'data']

    # Conversion to numpy arrays in HWC format (not Tensor!)
    images = images.reshape(-1, 32, 32, 3).astype(np.uint8)  # Shape (N,32,32,3), value range 0-255
    # Define test preprocessing (includes ToTensor and Normalize)
    transform_test = transforms.Compose([
        transforms.ToTensor(),  # Automatically convert numpy to Tensor + normalize to [0,1]
        transforms.Normalize((0.4914, 0.4822, 0.4465),
                             (0.2023, 0.1994, 0.2010))
    ])

    # Apply preprocessing (be a numpy array)
    transformed_images = []
    for img in images:
        transformed_img = transform_test(img)
        transformed_images.append(transformed_img)

    return torch.stack(transformed_images)   # Merge all the images and the output shape should be (N, 3, 32, 32)


test_images = load_test_data()
print("Test images shape:", test_images.shape)

test_dataset = torch.utils.data.TensorDataset(test_images)
test_loader = DataLoader(test_dataset, batch_size=100, shuffle=False)

for batch in test_loader:
    images = batch[0]
    print("Batch images shape:", images.shape)
    break

# Loading Models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LightResNet(num_classes=10).to(device)

model.load_state_dict(torch.load("cifar10_resnet.pth", map_location=device))
model.eval()

# inference
y_preds = []
with torch.no_grad():
    for batch in test_loader:
        images = batch[0].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        y_preds.extend(predicted.cpu().numpy())

# Generating Submission Files
submission_df = pd.DataFrame({'ID': np.arange(len(y_preds)), 'Labels': y_preds})
submission_df.to_csv("submission.csv", index=False)
print("✅ CSV file generated: submission.csv")


Mounted at /content/drive
Test images shape: torch.Size([10000, 3, 32, 32])
Batch images shape: torch.Size([100, 3, 32, 32])


<ipython-input-3-2eba86d19389>:130: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("cifar10_resnet.pth", map_location=device))


RuntimeError: Error(s) in loading state_dict for LightResNet:
	Unexpected key(s) in state_dict: "layer1.2.conv1.weight", "layer1.2.bn1.weight", "layer1.2.bn1.bias", "layer1.2.bn1.running_mean", "layer1.2.bn1.running_var", "layer1.2.bn1.num_batches_tracked", "layer1.2.conv2.weight", "layer1.2.bn2.weight", "layer1.2.bn2.bias", "layer1.2.bn2.running_mean", "layer1.2.bn2.running_var", "layer1.2.bn2.num_batches_tracked", "layer1.2.se.fc.0.weight", "layer1.2.se.fc.0.bias", "layer1.2.se.fc.2.weight", "layer1.2.se.fc.2.bias", "layer1.0.se.fc.0.weight", "layer1.0.se.fc.0.bias", "layer1.0.se.fc.2.weight", "layer1.0.se.fc.2.bias", "layer1.1.se.fc.0.weight", "layer1.1.se.fc.0.bias", "layer1.1.se.fc.2.weight", "layer1.1.se.fc.2.bias", "layer2.2.conv1.weight", "layer2.2.bn1.weight", "layer2.2.bn1.bias", "layer2.2.bn1.running_mean", "layer2.2.bn1.running_var", "layer2.2.bn1.num_batches_tracked", "layer2.2.conv2.weight", "layer2.2.bn2.weight", "layer2.2.bn2.bias", "layer2.2.bn2.running_mean", "layer2.